In [1]:
from dataprocessing.processors.refined_ingestion import TrackingDataRefinedProcess, TimetableRefinedProcess, BusStopRefinedProcess, LineRefinedProcess
from dataprocessing.processors.trust_ingestion import TrustProcessing

from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

In [2]:
# from dataprocessing.processors.sparketl import ETLSpark

# etl_spark = ETLSpark()

# df = etl_spark.sqlContext.read.parquet("/data/trusted/vehicles").withColumn("hour", F.hour(F.col("event_timestamp"))).sort(F.asc("event_timestamp"))

In [13]:
%%time
TrustProcessing("2019-05")()

FILE: Column<b'input_file_name()'>
FILE: Column<b'input_file_name()'>
FILE: Column<b'input_file_name()'>
CPU times: user 42.7 ms, sys: 13.6 ms, total: 56.3 ms
Wall time: 37.6 s


In [3]:
%%time
LineRefinedProcess(2019,5,6)()
TimetableRefinedProcess(2019,5,6)()
BusStopRefinedProcess(2019,5,6)()
TrackingDataRefinedProcess(2019,5,6)()

CPU times: user 1.18 s, sys: 495 ms, total: 1.67 s
Wall time: 5min 21s


## AREA 51

In [3]:
trd = TrackingDataRefinedProcess(2019,5,2)

vehicles = trd.compute_metrics()

stop_events = trd.stop_events(vehicles)

In [4]:
stop_events.filter("vehicle = 'BN997'").sort(F.asc("stop_timestamp")).show(50)

+---------+--------+-------+-------------+-------------------+----------+----------+------------+----+-----+---+----+
|line_code|line_way|vehicle|moving_status|     stop_timestamp|  latitude| longitude|avg_velocity|year|month|day|hour|
+---------+--------+-------+-------------+-------------------+----------+----------+------------+----+-----+---+----+
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:21:00|-25.427685|-49.264708|        9.83|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:23:24|-25.432431|-49.262705|         6.0|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:24:07|-25.433933| -49.26203|        7.18|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:25:13| -25.43442|-49.262871|         7.2|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:27:33|-25.436813|-49.268886|        7.82|2019|    5|  2|   6|
|      001| Horario|  BN997|      STOPPED|2019-05-02 06:

In [5]:
bus_event_edges = trd.event_edges(vehicles)

events = trd.process_events(stop_events, bus_event_edges)

In [9]:
events.filter("vehicle = 'BN997'").sort(F.asc("event_timestamp")).toPandas().to_csv("events.csv",index=False)

In [7]:
events.count()

499691